#### Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

- MAR@k
- F-Measure

#### Доделать все функции, где стоит комментарий "сделать дома"

In [1]:
import numpy as np

In [23]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
prices_recommended = [100, 50, 200, 300, 150, 70, 80, 130, 170, 200]
prices_bought = [80, 90, 120, 290]

##### Hit rate

In [3]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

In [4]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [5]:
hit_rate(recommended_list, bought_list)

1

In [6]:
hit_rate_at_k(recommended_list, bought_list)

1

##### Precision

In [15]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [16]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [17]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [18]:
precision(recommended_list, bought_list)

0.2

In [19]:
precision_at_k(recommended_list, bought_list)

0.4

In [20]:
money_precision_at_k(recommended_list, bought_list, prices_recommended)

0.5

##### Recall

In [7]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

In [8]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [9]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    
    top_recommended_list = np.array(recommended_list)[:k]
    top_prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(top_recommended_list, bought_list)
    return np.dot(flags, top_prices_recommended).sum() / prices_bought.sum()
    
    

In [10]:
recall(recommended_list, bought_list)

0.5

In [11]:
recall_at_k(recommended_list, bought_list)

0.5

In [12]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)

0.6896551724137931

##### AP@k

In [21]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  

In [22]:
ap_k(recommended_list, bought_list)

0.75

##### MAP@k

In [27]:
recommended_list_users = [[1,2,5,9,3,4], [2,1,8,9,5,7]]
bought_list_users = [[2,8,7], [3,2,6]]

In [28]:
def map_k(recommended_list, bought_list, k=5):
    apk_sum = 0
    count_user = 0
    
    for rec_list_user, bought_list_user in zip(recommended_list, bought_list):
        apk_sum = apk_sum + ap_k(rec_list_user, bought_list_user, k)
        count_user = count_user + 1
    
    return apk_sum / count_user

In [29]:
map_k(recommended_list_users, bought_list_users)

0.75

##### NDCG@k

In [30]:
import math

In [35]:
def dcg(flags):
    sum = 0
    count = 0
    
    for index, flag in enumerate(flags):
        if index < 2:
            sum = sum + (flag / (index + 1))
        else:
            sum = sum + (flag / math.log(index + 1))
            
        count = count + 1
        
    return sum / count

In [36]:
def ideal_dcg(flags):
    sum = 0
    count = 0
    
    for index, flag in enumerate(flags):
        if index < 2:
            sum = sum + (1 / (index + 1))
        else:
            sum = sum + (1 / math.log(index + 1))
            
        count = count + 1
        
    return sum / count

In [37]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    top_recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(recommended_list, bought_list)
    
    value_dcg = dcg(flags) 
    value_ideal_dcg = ideal_dcg(flags)
    
    return value_dcg / value_ideal_dcg

In [38]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.2778498752476734

##### MRR@k

In [39]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    for index, value in enumerate(recommended_list[:k]):
        if value in bought_list:
            return 1/(index + 1)

In [40]:
def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    sum = 0
    count = 0
    
    for rec_list_user, bought_list_user in zip(recommended_list, bought_list):
        sum = sum + reciprocal_rank(rec_list_user, bought_list_user)
        count = count + 1
        
    return sum / count

In [41]:
mean_reciprocal_rank(recommended_list_users, bought_list_users)

0.75